# `AzureAIFaceAnalysisTool`

## Overview

### Integration details

>The `AzureAIFaceAnalysisTool` acts as a wrapper around the Azure AI Face Service/API. The Tool will detect faces is in an image 
>and can supply the position of faces, as well as important features like eyes, pupils, head pose etc.

### Tool features

This integration allows for the detection of faces using Azure's Face API. It supports various different models, but defaults to the latest. Furthermore, the user can configure which information they would like to be returned such as position of the faces and image quality of the faces.

## Setup

This section provides details about how the Azure AI Face integration works, including setup.

In [ ]:
import os

from langchain import hub

We will use a prompt to instruct the model. LangChain prompts can be configured, but for simplicity, we will use a premade prompt from LangSmith. This requires an API key, which can be set up here after registration.

In [ ]:
LANGSMITH_KEY = os.environ["LANGSMITH_KEY"]
prompt = hub.pull("hwchase17/structured-chat-agent", api_key=LANGSMITH_KEY)

Now we can use the `AzureAIFaceAnalysisTool` combined with a model, using `create_structured_chat_agent`.

In [ ]:
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_community.tools.azure_ai_services.face_analysis import (
    AzureAIFaceAnalysisTool,
)
from langchain_openai import AzureChatOpenAI

### Credentials

Credentials can be set by being passed as parameters and should be stored locally as environment variables.

In [ ]:
face_endpoint = os.environ["AZURE_AI_FACE_ENDPOINT"]
face_key = os.environ["AZURE_AI_FACE_KEY"]

## Instatiation

Credentials can be passed directly, but they can also be retrieved automatically by the constructor if environment variables named `AZURE_AI_FACE_ENDPOINT` and `AZURE_AI_FACE_KEY` are set.

In [ ]:
face = AzureAIFaceAnalysisTool(
    azure_ai_face_key=face_key,
    azure_ai_face_endpoint=face_endpoint,
)

In [ ]:
tools = [face]

model = AzureChatOpenAI(
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    azure_deployment=os.environ["COMPLETIONS_MODEL"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
)

Creating an `AgentExecutor` chain allows a model to use tools to assist in it's response.

In [ ]:
agent = create_structured_chat_agent(model, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

## Invocation

Get an input file path

In [ ]:
from pathlib import Path

_THIS_DIR = Path(__file__).parents[2]
_EXAMPLE_DIR = _THIS_DIR / "example_data"
IMAGE = _EXAMPLE_DIR / "face.jpg"

In [ ]:
input: str = IMAGE

### [Invoke directly with args](/docs/concepts/#invoke-with-just-the-arguments)

Firstly, the tool can be used by directly passing input as an argument. However, this is discouraged as the tool is intended to be used in an executor chain.

In [ ]:
face._run(input)

### [Invoke with ToolCall](/docs/concepts/#invoke-with-toolcall)

By using `.invoke`, the model can be told what to do and assess if using the tools it was given would assist in it's response. This is the intended use.

In [ ]:
agent_executor.invoke(
    {"input": f"Can you check the following image for faces : {input}"}
)

## Chaining

When creating an `AgentExecutor` as described earlier, an execution chain is formed. The sequence of events will be printed using the given prompt, and actions will occur in a chain-like manner.

In [ ]:
agent = create_structured_chat_agent(model, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

## API reference

[Azure AI Face python SDK](https://learn.microsoft.com/python/api/overview/azure/ai-vision-face-readme?view=azure-python-preview) | [Azure AI Face Overview](https://learn.microsoft.com/azure/ai-services/computer-vision/overview-identity)